# Introduction

This notebook illustrates the relationship between power consumption and CPU load 
of a laptop computer. Sample power consumption and CPU load data is collected from 
a Macbook Pro with Core i5 2.9 GHZ CPU (4 cores) running on MacOS Seirra.

----
Tools used: 

**psutil**: to gather CPU load at user-defined sampling intervals

**MacOS SystemProfiler**: to collect battery discharge info. That is, 
                          voltage and current readings.
                          
**pandas and numpy**: to process the resulting data

**bokeh**: for plotting

In [112]:
import psutil
import subprocess
import time
import pandas as pd
import numpy as np

In [95]:
def cpuLoadPower(sampling_interval):
    cpu_load = []

    for sample in range(150):
        subprocess.check_call("./power.sh", shell=True)
        cpu_load.append(psutil.cpu_percent(sampling_interval))
        time.sleep(sampling_interval)
    return cpu_load
    

In [103]:
cpu_load = cpuLoadPower(5) # CPU load and Power consumption logging with 5 sec sampling interval

data = pd.read_csv('volt-amp.txt', sep='\t',header=None)
data.rename(columns={0:'Time', 1:'mV', 2: 'mA'}, inplace=True)
data['CPU'] = cpu_load
data['Power'] = data.mV * data.mA * -1e-6   # Power = V * A
data.Time = data.Time - data.Time[0]  # relative to the beginning time of sampling 

In [104]:
data.head(5)

,Time,mV,mA,CPU,Power
0,0,11170,-920,12.8,10.27640
1,11,11170,-920,4.7,10.27640
2,21,11184,-790,6.8,8.83536
3,31,11184,-790,4.9,8.83536
4,42,11184,-790,5.1,8.83536


In [105]:
data.tail(5)

,Time,mV,mA,CPU,Power
145,1520,10833,-1681,35.4,18.210273
146,1531,10833,-1681,31.5,18.210273
147,1541,10833,-1681,30.4,18.210273
148,1552,10833,-1681,31.8,18.210273
149,1562,10833,-1681,30.9,18.210273


In [53]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

In [54]:
output_notebook()

Loading BokehJS ...

In [55]:
from bokeh.models import LinearAxis, Range1d

In [108]:
p = figure(plot_width=600, plot_height=400, toolbar_location="above")

p.line(data['Time'], data['CPU'], color="firebrick", legend='CPU load')

p.extra_y_ranges = {"foo": Range1d(start=0, end=35)}
p.line(data['Time'], data['Power'], color="navy", y_range_name="foo", legend='Power Consumption')

p.add_layout(LinearAxis(y_range_name="foo"), 'right')


p.xaxis.axis_label = "Time (sec)"

p.yaxis[0].axis_label = "CPU load (%)"
p.yaxis[0].axis_label_text_color = "firebrick"
p.yaxis[0].major_label_text_color = "firebrick"
p.yaxis[0].major_tick_line_color = 'firebrick'
p.yaxis[0].minor_tick_line_color = 'firebrick'
p.yaxis[0].axis_line_color = 'firebrick'

p.yaxis[1].axis_label = "Power (Watt)"
p.yaxis[1].axis_label_text_color = "navy"
p.yaxis[1].major_label_text_color = "navy"
p.yaxis[1].major_tick_line_color = 'navy'
p.yaxis[1].minor_tick_line_color = 'navy'
p.yaxis[1].axis_line_color = 'navy'

#p.legend.location = "top_center"
p.legend.location = "top_right"
p.grid.grid_line_dash = [2,2]
p.grid.grid_line_alpha = 0.4
show(p)

The above plot shows that whenever CPU load hikes so does power consumption and vise versa.
Interestingly, there seems to be a consistent time gap between the change in CPU load and 
power consumption. 


In [141]:
fit = np.polyfit(data['CPU'], data['Power'], 1)
fn = np.poly1d(fit)


p = figure(plot_width=600, plot_height=400, toolbar_location="above")

p.circle(data['CPU'], data['Power'], color="firebrick", size=5)
p.line(data['CPU'], fn(data['CPU']), color="firebrick", legend="Power = 0.23*CPU_Load + 11.94")

p.xaxis.axis_label = "CPU LOAD (%)"
p.yaxis.axis_label = "POWER (Watt)"
p.legend.location = "bottom_right"
show(p)


The linear line in the above plot shows that the power consumption is about $11.94$ Watts <br /> 
even when the computer is idle. Of course, the CPU is not the only component that consumes power.

Power consumption ranges between $12$ and $35$ Watts as the CPU load goes from $0$ to $100%$.